# Getting started with HDF5 and PyTables

*11/03/2018 - Giacomo Debidda @PyCon Slovakia*

In [33]:
import os
import numpy as np
import pandas as pd
import tables as tb

In [4]:
np.set_printoptions(precision=2, suppress=True)

In [41]:
tb.__version__

'3.4.2'

In [43]:
tb.hdf5_version

'1.8.18'

In [47]:
tb.print_versions()

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
PyTables version:    3.4.2
HDF5 version:        1.8.18
NumPy version:       1.14.1
Numexpr version:     2.6.4 (not using Intel's VML/MKL)
Zlib version:        1.2.11 (in Python interpreter)
LZO version:         2.09 (Feb 04 2015)
BZIP2 version:       1.0.6 (6-Sept-2010)
Blosc version:       1.11.3 (2017-03-09)
Blosc compressors:   blosclz (1.0.5), lz4 (1.7.5), lz4hc (1.7.5), snappy (1.1.1), zlib (1.2.8), zstd (1.1.3)
Blosc filters:       shuffle, bitshuffle
Python version:      3.6.3 |Anaconda, Inc.| (default, Nov 20 2017, 20:41:42) 
[GCC 7.2.0]
Platform:            Linux-4.4.0-116-generic-x86_64-with-debian-stretch-sid
Byte-ordering:       little
Detected cores:      4
Default encoding:    utf-8
Default FS encoding: utf-8
Default locale:      (en_US, UTF-8)
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=


In [5]:
data_dir = os.path.join(os.getcwd(), 'data')
print(data_dir)

/home/jack/Repos/hdf5-pycon-slovakia/data


### HDF5: a filesystem in a file

HDF5 is a data model, library, and file format for storing and managing big and complex data.

An HDF5 file can be thought of as a container (or group) that holds a variety of heterogeneous data objects (or datasets). The datasets can be almost anything: images, tables, graphs, or even documents, such as PDF or Excel.

- Datasets (i.e. files in a filesystem)
- Groups (i.e. directories in a filesystem)
- Attributes (i.e. metadata of file/directory)

![HDF5 structure](img/hdf5_structure.jpg)

Working with groups and group members is similar to working with directories and files in UNIX.

**/** root group (every HDF5 file has a root group)

**/foo** member of the root group called foo

**/foo/bar** member of the group foo called bar

### HDF5 in the Python data stack

![h5py - PyTables refactor](img/h5py-pytables-refactor.png)

![PyTables logo](img/pytables-logo.png)

- Does not want to be a complete wrapper for the entire HDF5 C API
- High level abstraction over HDF5 (it's more "battery included" than h5py)
- Does not depend on h5py (at the moment)
- Natural naming
- Fast searches (indexing, out-of-core querying)
- Built-in compression
- Undo mode

### PyTables provides high-level abstractions over the HDF5 Dataset

Homogenous dataset:

- **Array**
- **CArray**
- **EArray**
- **VLArray**

Heterogenous dataset:

- **Table**

In [80]:
arr = np.arange(10).astype('float32')
arr

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.], dtype=float32)

### Natural naming

PyTables nodes (i.e. datasets and groups in the HDF5 file) can be accessed with the dot notation.

In [7]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    f.create_array(where='/', 
                   name='my array',
                   title='My PyTables Array',
                   obj=arr)

/home/jack/.virtualenvs/hdf5-pycon-slovakia/lib/python3.6/site-packages/tables/path.py:112: NaturalNameWarning: object name is not a valid Python identifier: 'my array'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


### Array

[Docs](http://www.pytables.org/usersguide/libref/homogenous_storage.html#the-array-class)

- Fastest I/O speed
- Must fit in memory
- Not compressible
- Not enlargeable

In [15]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    f.create_array(where='/', 
                   name='my_array',
                   title='My PyTables Array',
                   obj=arr)

In [14]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='r') as f:
    print(f.root.my_array)

/my_array (Array(4,)) 'My PyTables Array'


In [9]:
!ptdump 'data/my_pytables_file.h5'

/ (RootGroup) ''
/my_array (Array(4,)) 'My PyTables Array'


In [7]:
!pttree --use-si-units --sort-by 'size' 'data/my_pytables_file.h5'


------------------------------------------------------------

/ (RootGroup)
`--my_array (Array)
      mem=32.0B, disk=32.0B [100.0%]

------------------------------------------------------------
Total branch leaves:    1
Total branch size:      32.0B in memory, 32.0B on disk
Mean compression ratio: 1.00
HDF5 file size:         2.2kB
------------------------------------------------------------



### CArray

[Docs](http://www.pytables.org/usersguide/libref/homogenous_storage.html#carrayclassdescr)

- Compressible, chunked storage
- Not enlargeable

In [67]:
filters = tb.Filters(complevel=5, complib='zlib')

Tips on how to use compression (from the PyTables docs)

- A mid-level (5) compression is sufficient. No need to go all the way up (9)
- Use zlib if you must guarantee complete portability
- Use blosc all other times (it is optimized for HDF5)

In [ ]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    f.create_carray(
        where='/',
        name='my_carray',
        title='My PyTables CArray'
        obj=arr,
        filters=filters)

### EArray

[Docs](http://www.pytables.org/usersguide/libref/homogenous_storage.html#earrayclassdescr)

- Enlargeable on **one** dimension (append)
- Compressible

In [75]:
# One (and only one) of the shape dimensions *must* be 0.
# The dimension being 0 means that the resulting EArray object can be extended along it.
# Multiple enlargeable dimensions are not supported (at the moment).
num_columns = 5
shape = (0, num_columns)

with tb.open_file(filename='data/my_pytables_file.h5', mode='w') as f:
    # you can create an EArray and fill it later, but you need to specify atom and shape
    f.create_earray(
        where='/',
        name='my_earray',
        title='My PyTables EArray',
        atom=tb.Float32Atom(),
        shape=shape,
        filters=filters)

In [76]:
num_rows = 1000000  # 1 million
matrix = np.random.random((num_rows, num_columns)).astype('float32')

In [77]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='a') as f:
    earray = f.root.my_earray
    earray.append(sequence=matrix[0:10, :])

In [78]:
with tb.open_file(filename='data/my_pytables_file.h5', mode='a') as f:
    earray = f.root.my_earray
    earray.append(sequence=matrix[11:50, :])

### VLArray

### Table

### Introducing a real dataset: NYC yellow taxi dataset

Without a real world example I find it hard to reason about...
http://www.nyc.gov/html/tlc/html/about/trip_record_data.shtml

In [12]:
!less 'data/taxi+_zone_lookup.csv'

"LocationID","Borough","Zone","service_zone"
1,"EWR","Newark Airport","EWR"
2,"Queens","Jamaica Bay","Boro Zone"
3,"Bronx","Allerton/Pelham Gardens","Boro Zone"
4,"Manhattan","Alphabet City","Yellow Zone"
5,"Staten Island","Arden Heights","Boro Zone"
6,"Staten Island","Arrochar/Fort Wadsworth","Boro Zone"
7,"Queens","Astoria","Boro Zone"
8,"Queens","Astoria Park","Boro Zone"
9,"Queens","Auburndale","Boro Zone"
10,"Queens","Baisley Park","Boro Zone"
11,"Brooklyn","Bath Beach","Boro Zone"
12,"Manhattan","Battery Park","Yellow Zone"
13,"Manhattan","Battery Park City","Yellow Zone"
14,"Brooklyn","Bay Ridge","Boro Zone"
15,"Queens","Bay Terrace/Fort Totten","Boro Zone"
16,"Queens","Bayside","Boro Zone"
17,"Brooklyn","Bedford","Boro Zone"
18,"Bronx","Bedford Park","Boro Zone"
19,"Queens","Bellerose","Boro Zone"
20,"Bronx","Belmont","Boro Zone"
21,"Brooklyn","Bensonhurst East","Boro Zone"
22,"Brooklyn","Bensonhurst West","Boro Zone"
:a/taxi+_zone_lookup.csv

In [13]:
!less 'data/yellow_tripdata_2017-12.csv'

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount

1,2017-12-01 00:12:00,2017-12-01 00:12:51,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8
1,2017-12-01 00:13:37,2017-12-01 00:13:47,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8
1,2017-12-01 00:14:15,2017-12-01 00:15:05,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8
1,2017-12-01 00:15:33,2017-12-01 00:15:37,1,.00,1,N,226,226,3,2.5,0.5,0.5,0,0,0.3,3.8
1,2017-12-01 00:50:03,2017-12-01 00:53:35,1,.00,1,N,145,145,2,4,0.5,0.5,0,0,0.3,5.3
1,2017-12-01 00:14:20,2017-12-01 00:28:35,1,4.20,1,N,82,258,2,15,0.5,0.5,0,0,0.3,16.3
1,2017-12-01 00:20:32,2017-12-01 00:31:24,1,5.40,1,N,50,116,2,17,0.5,0.5,0,0,0.3,18.3
1,2017-12-01 00:01:46,2017-12-01 00:12:19,1,1.90,1,N,161,107,1,9,0.5,0.5,2.05,0,0.3,12.35
1,2017-12-01 00:17:52,2017-12-01 00:32:35,1,3.30,1,N,107,263,1,12.5,0.5,0

In [50]:
# data dictionary for NY yellow taxi CSV files
# http://www.nyc.gov/html/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf
data_dictionary = {
    'VendorID': 'A code indicating the TPEP provider that provided the record',
    'tpep_pickup_datetime': 'The date and time when the meter was engaged ',
    'tpep_dropoff_datetime': 'The date and time when the meter was disengaged ',
    'passenger_count': 'The number of passengers in the vehicle',
    'trip_distance': 'The elapsed trip distance in miles reported by the taximeter',
    'PULocationID': 'A code indicating the zone + borough of th pickup location',
    'DOLocationID': 'A code indicating the zone + borough of th dropoff location',
    'payment_type': 'A numeric code signifying how the passenger paid for the trip',
    'fare_amount': 'The time-and-distance fare calculated by the meter',
    'tip_amount': 'Tip amount – This field is automatically populated for credit card tips. Cash tips are not included',
    'total_amount': 'The total amount charged to passengers. Does not include cash tips',
}

Table schema

In [18]:
class TaxiTableDescription(tb.IsDescription):
    vendor_id = tb.UInt8Col(pos=0)
    pickup_timestamp_ms = tb.Int64Col()
    dropoff_timestamp_ms = tb.Int64Col()
    passenger_count = tb.UInt8Col()
    trip_distance = tb.Float32Col()
    pickup_location_id = tb.UInt16Col()
    dropoff_location_id = tb.UInt16Col()
    payment_type = tb.UInt8Col()
    fare_amount = tb.Float32Col()
    tip_amount = tb.Float32Col()
    total_amount = tb.Float32Col()

In [28]:
h5_file_path = os.path.join(data_dir, 'NYC-yellow-taxis-100k.h5')
print(h5_file_path)

/home/jack/Repos/hdf5-pycon-slovakia/data/NYC-yellow-taxis-100k.h5


In [19]:
filters = tb.Filters(complevel=5, complib='zlib')

with tb.open_file(filename=h5_file_path, mode='w') as f:
    f.create_table(
        where='/',
        name='yellow_taxis_2017_12',
        description=TaxiTableDescription,
        title='NYC Yellow Taxi data December 2017',
        filters=filters)
    # add metadata
    table_where = '/yellow_taxis_2017_12'
    for key, val in data_dictionary.items():
        f.set_node_attr(where=table_where, attrname=key, attrvalue=val)

In [26]:
!ptdump 'data/NYC-yellow-taxis-100k.h5'  # try also h5dump

/ (RootGroup) ''
/yellow_taxis_2017_12 (Table(0,), shuffle, zlib(5)) 'NYC Yellow Taxi data December 2017'


In [32]:
def date_to_timestamp_ms(date_obj):
    timestamp_in_nanoseconds = date_obj.astype('int64')
    timestamp_in_ms = (timestamp_in_nanoseconds / 1000000).astype('int64')
    return timestamp_in_ms

def fill_table(table, mapping, df):
    num_records = df.shape[0]  # it's equal to the chunksize used in read_csv
    row = table.row
    for i in range(num_records):
        row['vendor_id'] = df[mapping['vendor_id']].values[i]

        pickup_ms = date_to_timestamp_ms(df[mapping['pickup_datetime']].values[i])
        row['pickup_timestamp_ms'] = pickup_ms
        dropoff_ms = date_to_timestamp_ms(df[mapping['dropoff_datetime']].values[i])
        row['dropoff_timestamp_ms'] = dropoff_ms

        row['passenger_count'] = df['passenger_count'].values[i]
        row['trip_distance'] = df['trip_distance'].values[i]

        row['pickup_location_id'] = df['PULocationID'].values[i]
        row['dropoff_location_id'] = df['DOLocationID'].values[i]

        row['fare_amount'] = df['fare_amount'].values[i]
        row['tip_amount'] = df['tip_amount'].values[i]
        row['total_amount'] = df['total_amount'].values[i]

        row['payment_type'] = df['payment_type'].values[i]
        row.append()
    table.flush()

*Remember to flush:* Remember, flushing a table is a very important step as it will not only help to maintain the integrity of your file, but also will free valuable memory resources (i.e. internal buffers) that your program may need for other things.

In [62]:
%%time
# Open the HDF5 file in 'a'ppend mode and populate the table with CSV data
with tb.open_file(filename=h5_file_path, mode='a') as f:
    # Left, the key we want to use. Right, the key in the CSV file
    mapping = {
        'vendor_id': 'VendorID',
        'pickup_datetime': 'tpep_pickup_datetime',
        'dropoff_datetime': 'tpep_dropoff_datetime',
        'pickup_location_id': 'PULocationID',
        'dropoff_location_id': 'DOLocationID'
    }

    # define the dtype to use when reading the CSV with pandas (this has nothing to do with the HDF5 table)
    dtype = {'VendorID': 'category', 'payment_type': 'category'}
    parse_dates = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']

    table = f.get_node(where='/yellow_taxis_2017_12')
 
    csv_file_path = os.path.join(data_dir, 'yellow_tripdata_2017-12.csv')

    # read in chunks because these CSV files are too big
    chunksize = 100000
    for chunk in pd.read_csv(
        csv_file_path, chunksize=chunksize, dtype=dtype,
        skipinitialspace=True, parse_dates=parse_dates):
        df = chunk.reset_index(drop=True)
        fill_table(table, mapping, df)
        # remove the break statement to process all chunks (it will take ~20 minutes)
        break

CPU times: user 10.3 s, sys: 12 ms, total: 10.3 s
Wall time: 10.3 s


In [46]:
tb.is_pytables_file('data/NYC-yellow-taxis-100k.h5')

'2.1'

In [65]:
help(table.remove_rows)

Help on method remove_rows in module tables.table:

remove_rows(start=None, stop=None, step=None) method of tables.table.Table instance
    Remove a range of rows in the table.
    
    If only start is supplied, that row and all following will be deleted.
    If a range is supplied, i.e. both the start and stop parameters are
    passed, all the rows in the range are removed.
    
    .. versionchanged:: 3.0
       The start, stop and step parameters now behave like in slice.
    
    .. seealso:: remove_row()
    
    Parameters
    ----------
    start : int
        Sets the starting row to be removed. It accepts negative values
        meaning that the count starts from the end.  A value of 0 means the
        first row.
    stop : int
        Sets the last row to be removed to stop-1, i.e. the end point is
        omitted (in the Python range() tradition). Negative values are also
        accepted. If None all rows after start will be removed.
    step : int
        The step size 

In [40]:
!pttree --use-si-units --sort-by 'size' 'data/NYC-yellow-taxis-100k.h5'


------------------------------------------------------------

/ (RootGroup)
`--yellow_taxis_2017_12 (Table)
      mem=7.8MB, disk=3.5MB [100.0%]

------------------------------------------------------------
Total branch leaves:    1
Total branch size:      7.8MB in memory, 3.5MB on disk
Mean compression ratio: 0.45
HDF5 file size:         3.5MB
------------------------------------------------------------



### Chunking

See https://github.com/tomkooij/scipy2017/blob/master/notebooks/03-Chunking.ipynb

### Compression

See https://github.com/tomkooij/scipy2017/blob/master/notebooks/04-Using-Compression.ipynb

### Expressions

See https://github.com/tomkooij/scipy2017/blob/master/notebooks/07-Expressions.ipynb

### Searches

### Indexes

### Subclass Table (and add validation)

### Create custom datatype

See https://github.com/tomkooij/scipy2017/blob/master/notebooks/02-Datatypes-in-HDF5.ipynb

### Links

soft links and hard links

### Add column to table (like a database schema migration)

### Filenode

### Undo/redo

### HDF5 users and job offers

### Where to go from here